In [1]:
import numpy as np
import pandas as pd
import os
import sys
import metrics as mt

In [2]:
split_char=","
sep_char="\t"
test_set_path = '../Dataset/test_set.csv'
sub_path = '../submissions/'
test_set = pd.read_csv(test_set_path, sep=sep_char,index_col=0)
test_set = test_set['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(split_char)] if x!='' else []))
#test_set = test_set.fillna(int(0))

In [8]:
sub_name = 'top_pop.csv'

submission = pd.read_csv(os.path.join(sub_path, sub_name), sep=sep_char,index_col=0)
submission = submission.fillna('')
submission.columns = ['items']
sub_ranks = submission['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(split_char)] if x!='' else []))
#sub_ranks = sub_ranks.fillna(0)

In [9]:
print('Running evaluation over {} users'.format(len(test_set)))
print('Your submission contains predictions for {} of them'.format(submission.shape[0]))
metrics = mt.compute_metrics(sub_ranks,test_set,k=5)
print('Average Recall: '), metrics[0]
print('Mean average precision at 2: '), metrics[2]
print('Mean average precision at 5: '), metrics[3]
print('Mean average precision at 10: '), metrics[4]
print('Mean average precision at 20: '), metrics[5]
print('Average user success: '), metrics[1]
print('Normalized Discount Cumulative Gain: '), metrics[6]
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

Running evaluation over 10000 users
Your submission contains predictions for 10000 of them
Average Recall:  0.0311974332891
Mean average precision at 2:  0.0062
Mean average precision at 5:  0.00578572222222
Mean average precision at 10:  0.00631907440476
Mean average precision at 20:  0.00651285835925
Average user success:  0.0606
Normalized Discount Cumulative Gain:  0.014136201952
Competition score:  21921.4866578


In [5]:
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

Competition score:  21921.4866578


In [6]:
submission = pd.read_csv(test_set_path, sep=sep_char,index_col=0)
submission = submission.fillna('')
submission.columns = ['items']
sub_ranks = submission['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(split_char)] if x!='' else []))
#sub_ranks = sub_ranks.fillna(int(0))
print('PERFECT SCORE:')
metrics = mt.compute_metrics(sub_ranks,test_set,k=30)
print('Average Recall: '), metrics[0]
print('Mean average precision at 2: '), metrics[2]
print('Mean average precision at 5: '), metrics[3]
print('Mean average precision at 10: '), metrics[4]
print('Mean average precision at 20: '), metrics[5]
print('Average user success: '), metrics[1]
print('Normalized Discount Cumulative Gain: '), metrics[6]
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

PERFECT SCORE:
Average Recall:  1.0
Mean average precision at 2:  1.0
Mean average precision at 5:  1.0
Mean average precision at 10:  1.0
Mean average precision at 20:  1.0
Average user success:  1.0
Normalized Discount Cumulative Gain:  1.0
Competition score:  740106.0


Prove
--

In [36]:
import math
def dcg(recommended_items,relevant_items):
    is_relevant = np.in1d(recommended_items[:30], relevant_items, assume_unique=True).astype(np.float32)[:30]
    pos = 1
    gain = 0
    for i in is_relevant:
        gain+= (math.pow(2,i)-1)/math.log(pos+1)
        pos+=1
    return gain

def ndcg(recommended_items,relevant_items):
    return dcg(recommended_items,relevant_items)/float(dcg(relevant_items,relevant_items))

uid = 302
recommended_items = sub_ranks.ix[uid].values
relevant_items = test_set.ix[uid].values   

print dcg(recommended_items,relevant_items)
print ndcg(recommended_items,relevant_items)

6.0
4.76762569144
1.0


In [23]:
math.log(1,2)

0.0

In [7]:
#Metrics
import numpy as np
import pandas as pd

def ap_at_k(recommended_items, relevant_items, k):
    is_relevant = np.in1d(recommended_items[:k], relevant_items, assume_unique=True).astype(np.float32)
    total_relevant = len(relevant_items[~np.isnan(relevant_items)])
    score = np.sum(np.cumsum(is_relevant) * is_relevant / np.arange(1, k+1)) / min(k, total_relevant)
    assert 0 <= score <= 1
    return score

def precision_at_k(recommended_items, relevant_items, k):
	return float(np.intersect1d(recommended_items[:k], relevant_items).shape[0]) / k

def recall(recommended_items, relevant_items):
    if relevant_items.shape[0] == 0: 
        return 0.0
    total_relevant = len(relevant_items[~np.isnan(relevant_items)])
    return float(np.intersect1d(recommended_items[:30], relevant_items).shape[0]) / min(30, total_relevant)

def user_success(recommended_items, relevant_items):
	return float(np.intersect1d(recommended_items[:30], relevant_items).shape[0] > 0)

def compute_metrics(S, T, k=30):
    recall_avg, user_success_avg, map_at_2, map_at_5, map_at_10, map_at_20 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    for uid in S.index:
        #parse ids
        recommended_items = S.ix[uid].values
        relevant_items = T.ix[uid].values    
        
        #compute metrics for a user
        prec = precision_at_k(recommended_items, relevant_items, 5)
        rec = recall(recommended_items, relevant_items)      
        usucc = user_success(recommended_items, relevant_items)
        
        #update averages
        recall_avg += rec
        user_success_avg += usucc
        
        map_at_2 += ap_at_k(recommended_items, relevant_items, 2)
        map_at_5 += ap_at_k(recommended_items, relevant_items, 5)
        map_at_10 += ap_at_k(recommended_items, relevant_items, 10)
        map_at_20 += ap_at_k(recommended_items, relevant_items, 20)
        
        
    recall_avg /= S.shape[0]
    user_success_avg /= S.shape[0]
    map_at_2 /= S.shape[0]
    map_at_5 /= S.shape[0]
    map_at_10 /= S.shape[0]
    map_at_20 /= S.shape[0]
    
    return (recall_avg, user_success_avg, map_at_2, map_at_5, map_at_10, map_at_20)

In [8]:
sub_ranks

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
user_id,,,,,,,,,,,,,,,,,,,,,
242,794156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,2816087,405029,386805,224811,1501340,281913,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,389202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,2755404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
927,621104,2386634,1688480,2200671,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,2086845,158270,861853,51035,1891281,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,1252522,26793,159660,1073391,167049,1325179,293384,1057330,1859201,866523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1509,343132,1686571,388915,21586,828489,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1797,1652172,888658,1211336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
